In [1]:
import numpy as np
import pandas as pd

In [2]:
train_seq = pd.read_csv('train/train_text_seq.csv')
valid_seq = pd.read_csv('valid/valid_text_seq.csv')

In [3]:
train_str = train_seq['input_str']
train_label = train_seq['label']
valid_str = valid_seq['input_str']
valid_label = valid_seq['label']

In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Perform one-hot encoding on the training data
encoder = OneHotEncoder()
train_encoded = encoder.fit_transform(train_str.str[:50].values.reshape(-1, 1))

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(train_encoded, train_label, test_size=0.2, random_state=42)

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier()

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)


RandomForestClassifier()

In [5]:
accuracy = rf_classifier.score(X_valid, y_valid)
print("Accuracy:", accuracy)


Accuracy: 0.507768361581921


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Example dataset (replace with your real data)
data = pd.read_csv('train/train_text_seq.csv')

# One-hot encoding for digits 0-9
def one_hot_encode_strings(strings):
    # Convert the string to a list of characters and reshape to (-1, 1) to prepare for one-hot encoding
    strings_as_digits = np.array([[int(digit) for digit in list(s)] for s in strings])
    
    # Use OneHotEncoder from sklearn
    encoder = OneHotEncoder(categories=[range(10)] * 50, sparse_output=False)
    encoded = encoder.fit_transform(strings_as_digits)
    
    return encoded

# Prepare the data
X = one_hot_encode_strings(data['input_str'])
y = data['label'].values

# Split data into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Example model - Random Forest Classifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.6370


In [10]:
d= pd.read_csv('valid/valid_text_seq.csv')
X_test = one_hot_encode_strings(d['input_str'])
y_test = d['label'].values

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy:.4f}')


Epoch 1/10


142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5027 - loss: 0.7001 - val_accuracy: 0.5869 - val_loss: 0.6794
Epoch 2/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.6675 - loss: 0.6331 - val_accuracy: 0.5993 - val_loss: 0.6520
Epoch 3/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.7576 - loss: 0.5227 - val_accuracy: 0.6390 - val_loss: 0.6230
Epoch 4/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - accuracy: 0.8432 - loss: 0.3988 - val_accuracy: 0.6505 - val_loss: 0.6437
Epoch 5/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - accuracy: 0.9125 - loss: 0.2746 - val_accuracy: 0.6443 - val_loss: 0.7466
Epoch 6/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step - accuracy: 0.9600 - loss: 0.1569 - val_accuracy: 0.6602 - val_loss: 0.8643
Epoch 7/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - accuracy: 0.9854 - loss: 0.0810 - val_accuracy: 0.6531 - val_loss: 1.0687
Epoch 8/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.9965 - loss: 0.0371 - val_accuracy:

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

# Assuming X_train contains sequences of 50 digits (as integers) and y_train contains binary labels (0/1)
# For embedding, we assume the digits are integers between 0 and 9

# Hyperparameters
max_length = 50  # Each string has 50 digits
vocab_size = 10  # Digits from 0-9
embedding_dim = 8  # Size of the dense vector for each digit

# Create the model
model = Sequential()

# Embedding layer - maps each digit to an 8-dimensional vector
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))

# Add a 1D Convolutional layer with 32 filters, kernel size of 3, and ReLU activation
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))

# Add Batch Normalization
model.add(BatchNormalization())

# Max pooling layer to reduce dimensionality
model.add(MaxPooling1D(pool_size=2))

# Another Conv1D layer to capture more features
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten the output to feed into a dense layer
model.add(Flatten())

# Dense layer with 64 neurons and ReLU activation
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.3))

# Output layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
142/142 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4973 - loss: 0.6995 - val_accuracy: 0.5031 - val_loss: 0.6931
Epoch 2/25
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5495 - loss: 0.6890 - val_accuracy: 0.5552 - val_loss: 0.6915
Epoch 3/25
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6042 - loss: 0.6648 - val_accuracy: 0.5799 - val_loss: 0.6847
Epoch 4/25
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6384 - loss: 0.6368 - val_accuracy: 0.6178 - val_loss: 0.6602
Epoch 5/25
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6616 - loss: 0.6045 - val_accuracy: 0.6267 - val_loss: 0.6496
Epoch 6/25
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6889 - loss: 0.5757 - val_accuracy: 0.6311 - val_loss: 0.6444
Epoch 7/25
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7200 - loss: 0.5434 - val_accuracy: 0.6381 - val_loss: 0.6489
Epoch 8/25
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7492 - loss: 0.5045 - val_accuracy: 0.

In [30]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.6524


In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Hyperparameters
max_length = 50  # Each string has 50 digits
vocab_size = 10  # Digits from 0-9
embedding_dim = 16  # Increased embedding dimension for richer representation
l2_reg = 0.001  # L2 regularization parameter

# Create the model
model = Sequential()

# Embedding layer - maps each digit to a dense vector
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))

# Convolutional layers to capture local patterns
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu', kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

# Bidirectional LSTM to capture forward and backward dependencies
model.add(Bidirectional(LSTM(128, return_sequences=False, kernel_regularizer=l2(l2_reg))))
model.add(BatchNormalization())

# Dense layer with L2 regularization and dropout
model.add(Dense(64, activation='relu', kernel_regularizer=l2(l2_reg)))
model.add(Dropout(0.3))

# Output layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with Adam optimizer and learning rate scheduler
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model with callbacks
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[reduce_lr])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/50


71/71 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.4984 - loss: 1.3888 - val_accuracy: 0.5031 - val_loss: 1.2064 - learning_rate: 0.0010
Epoch 2/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.5327 - loss: 1.1963 - val_accuracy: 0.5031 - val_loss: 1.1611 - learning_rate: 0.0010
Epoch 3/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - accuracy: 0.5326 - loss: 1.1024 - val_accuracy: 0.5031 - val_loss: 1.2889 - learning_rate: 0.0010
Epoch 4/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.5394 - loss: 1.0245 - val_accuracy: 0.5031 - val_loss: 1.4675 - learning_rate: 0.0010
Epoch 5/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - accuracy: 0.5668 - loss: 0.9552 - val_accuracy: 0.5031 - val_loss: 1.9256 - learning_rate: 0.0010
Epoch 6/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.5733 - loss: 0.9113 - val_accuracy: 0.5031 - val_loss: 2.4006 - learning_rate: 5.0000e-04
Epoch 7/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.5850 - loss: 0.8775 - val_accuracy: 